In [1]:
import set_working_directory

In [2]:
from cogent3 import open_data_store

dstore = open_data_store("data/raw.zip", suffix="fa", mode="r")
print(dstore)

1035x member ReadOnlyDataStoreZipped(source='/Users/gavin/repos/Cogent3/doc/data/raw.zip', members=[DataMember(data_store=/Users/gavin/repos/Cogent3/doc/data/raw.zip, unique_id=ENSG00000157184.fa), DataMember(data_store=/Users/gavin/repos/Cogent3/doc/data/raw.zip, unique_id=ENSG00000131791.fa)]...)


In [3]:
dstore.describe

completed,1035
not_completed,0
logs,0


In [4]:
m = dstore[0]
m

DataMember(data_store=/Users/gavin/repos/Cogent3/doc/data/raw.zip, unique_id=ENSG00000157184.fa)

In [5]:
for m in dstore[:5]:
    print(m)

ENSG00000157184.fa
ENSG00000131791.fa
ENSG00000127054.fa
ENSG00000067704.fa
ENSG00000182004.fa


In [6]:
m.read()[:20] # truncating

'>Human\nATGGCGTACCGTG'

In [7]:
dstore = open_data_store("data/demo-locked.sqlitedb")
dstore.describe

completed,175
not_completed,0
logs,1


In [8]:
dstore.unlock(force=True)

In [9]:
dstore.summary_logs

2019-07-24 14:42:56,logs/load_unaligned-progressive_align-write_db-pid8650.log,3.7.3,gavin,/Users/gavin/miniconda3/envs/c3dev/lib/python3.7/site-packages/ipykernel_launcher.py -f /Users/gavin/Library/Jupyter/runtime/kernel-5eb93aeb-f6e0-493e-85d1-d62895201ae2.json,"load_unaligned(type='sequences', moltype='dna', format='fasta') + progressive_align(type='sequences', model='HKY85', gc=None, param_vals={'kappa': 3}, guide_tree=None, unique_guides=False, indel_length=0.1, indel_rate=1e-10) + write_db(type='output', data_path='../data/aligned-nt.tinydb', name_callback=None, create=True, if_exists='overwrite', suffix='json')"


In [10]:
dstore.logs

[DataMember(data_store=/Users/gavin/repos/Cogent3/doc/data/demo-locked.sqlitedb, unique_id=logs/load_unaligned-progressive_align-write_db-pid8650.log)]

In [11]:
print(dstore.logs[0].read()[:225]) # truncated for clarity

2019-07-24 14:42:56	Eratosthenes.local:8650	INFO	system_details : system=Darwin Kernel Version 18.6.0: Thu Apr 25 23:16:27 PDT 2019; root:xnu-4903.261.4~2/RELEASE_X86_64
2019-07-24 14:42:56	Eratosthenes.local:8650	INFO	python


In [12]:
from cogent3 import open_data_store

in_dstore = open_data_store("data/raw.zip", suffix="fa")

In [13]:
out_dstore = open_data_store("translated.sqlitedb", mode="w")

In [14]:
from cogent3 import get_app

load = get_app("load_unaligned", moltype="dna")
translate = get_app("translate_seqs")
write = get_app("write_db", data_store=out_dstore)
app = load + translate + write
app

load_unaligned(moltype='dna', format='fasta') + translate_seqs(moltype='dna',
gc=1, allow_rc=False, trim_terminal_stop=True) +
write_db(data_store=DataStoreSqlite(source=/Users/gavin/repos/Cogent3/doc/translated.sqlitedb,
mode=Mode.w, limit=None, verbose=False), id_from_source=<function get_unique_id
at 0x177f79a80>, serialiser=to_primitive(convertor=<function as_dict at
0x177f967a0>) + pickle_it())

In [15]:
out_dstore = app.apply_to(in_dstore)

In [16]:
out_dstore.describe

completed,1025
not_completed,10
logs,1


In [17]:
out_dstore.validate()

Num md5sum correct,1035
Num md5sum incorrect,0
Num md5sum missing,0
Has log,True


In [18]:
out_dstore.summary_not_completed

ERROR,translate_seqs,'AlphabetError: unres...h not divisible by 3',10,"ENSG00000198938.fa, ENSG00000183291.fa, ..."


In [19]:
len(out_dstore.not_completed)

10

In [20]:
out_dstore.not_completed[:2]

[DataMember(data_store=/Users/gavin/repos/Cogent3/doc/translated.sqlitedb, unique_id=ENSG00000198938),
 DataMember(data_store=/Users/gavin/repos/Cogent3/doc/translated.sqlitedb, unique_id=ENSG00000183291)]

In [21]:
import pathlib

fn = pathlib.Path("translated.sqlitedb")
fn.unlink()